<a href="https://colab.research.google.com/github/vishnusnath447/BYOB/blob/main/byob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BACKEND CODE**

In [ ]:
!pip install streamlit
!pip install transformers flask torch
!pip uninstall compressed-tensors -y
!pip install compressed-tensors
!pip install flask flask-cors
!pip install pyngrok

In [ ]:
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os

# Load the model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Default data
default_tasks = {"tasks": []}
default_meetings = {"meetings": []}

# Load JSON data
def load_json(file_path, default_data):
    if not os.path.exists(file_path):
        save_json(file_path, default_data)
        return default_data
    with open(file_path, "r") as file:
        return json.load(file)

# Save JSON data
def save_json(file_path, data):
    with open(file_path, "w") as file:
        json.dump(data, file, indent=4)

# Load tasks and meetings data
tasks = load_json(tasks_file, default_tasks)
meetings = load_json(meetings_file, default_meetings)

# Get pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Get upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Generate response
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Flask API setup
app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    prompt = data.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400
    response = generate_response(prompt, tasks, meetings)
    return jsonify({'response': response})

# Create task
@app.route('/tasks', methods=['POST'])
def create_task():
    data = request.json
    description = data.get('description')
    if not description:
        return jsonify({'error': 'Description is required'}), 400
    new_task = {"description": description, "status": "pending"}
    tasks["tasks"].append(new_task)
    save_json(tasks_file, tasks)
    return jsonify({"message": "Task created successfully", "task": new_task}), 201

# Edit task
@app.route('/tasks/<int:task_id>', methods=['PUT'])
def edit_task(task_id):
    data = request.json
    description = data.get('description')
    status = data.get('status')

    if task_id >= len(tasks["tasks"]):
        return jsonify({'error': 'Task not found'}), 404

    task = tasks["tasks"][task_id]
    if description:
        task["description"] = description
    if status:
        task["status"] = status

    save_json(tasks_file, tasks)
    return jsonify({"message": "Task updated successfully", "task": task})

# Delete task
@app.route('/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    if task_id >= len(tasks["tasks"]):
        return jsonify({'error': 'Task not found'}), 404
    deleted_task = tasks["tasks"].pop(task_id)
    save_json(tasks_file, tasks)
    return jsonify({"message": "Task deleted successfully", "task": deleted_task})

# Create meeting
@app.route('/meetings', methods=['POST'])
def create_meeting():
    data = request.json
    title = data.get('title')
    date = data.get('date')
    time = data.get('time')
    agenda = data.get('agenda')
    is_recording = data.get('is_recording', False)

    if not title or not date or not time or not agenda:
        return jsonify({'error': 'Title, date, time, and agenda are required'}), 400

    new_meeting = {
        "title": title,
        "date": date,
        "time": time,
        "agenda": agenda,
        "is_recording": is_recording
    }
    meetings["meetings"].append(new_meeting)
    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting created successfully", "meeting": new_meeting}), 201

# Edit meeting
@app.route('/meetings/<int:meeting_id>', methods=['PUT'])
def edit_meeting(meeting_id):
    data = request.json
    title = data.get('title')
    date = data.get('date')
    time = data.get('time')
    agenda = data.get('agenda')
    is_recording = data.get('is_recording')

    if meeting_id >= len(meetings["meetings"]):
        return jsonify({'error': 'Meeting not found'}), 404

    meeting = meetings["meetings"][meeting_id]
    if title:
        meeting["title"] = title
    if date:
        meeting["date"] = date
    if time:
        meeting["time"] = time
    if agenda:
        meeting["agenda"] = agenda
    if is_recording is not None:
        meeting["is_recording"] = is_recording

    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting updated successfully", "meeting": meeting})

# Delete meeting
@app.route('/meetings/<int:meeting_id>', methods=['DELETE'])
def delete_meeting(meeting_id):
    if meeting_id >= len(meetings["meetings"]):
        return jsonify({'error': 'Meeting not found'}), 404
    deleted_meeting = meetings["meetings"].pop(meeting_id)
    save_json(meetings_file, meetings)
    return jsonify({"message": "Meeting deleted successfully", "meeting": deleted_meeting})

# Run Flask in a background thread
def run_server():
    app.run(host='0.0.0.0', port=5000)

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()


# **FRONTEND CODE**

In [ ]:
import streamlit as st
import requests

# Backend URL
BASE_URL = "http://127.0.0.1:5000"

# Helper functions for API calls
def get_response(endpoint, method="GET", data=None):
    url = f"{BASE_URL}/{endpoint}"
    try:
        if method == "GET":
            response = requests.get(url)
        elif method == "POST":
            response = requests.post(url, json=data)
        elif method == "PUT":
            response = requests.put(url, json=data)
        elif method == "DELETE":
            response = requests.delete(url)
        else:
            return None
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# Streamlit UI
st.title("Task & Meeting Manager")

# Tabs for Tasks and Meetings
tabs = st.tabs(["Tasks", "Meetings", "Chat"])

# Tasks Tab
with tabs[0]:
    st.header("Tasks")

    # Fetch tasks
    tasks_data = get_response("tasks", "GET")
    if "tasks" in tasks_data:
        tasks = tasks_data["tasks"]
        if tasks:
            st.subheader("Pending Tasks")
            for i, task in enumerate(tasks):
                st.write(f"{i + 1}. {task['description']} - {task['status']}")
                if st.button(f"Mark Complete", key=f"complete_{i}"):
                    task_update = {"status": "completed"}
                    get_response(f"tasks/{i}", "PUT", task_update)
                    st.experimental_rerun()
                if st.button(f"Delete Task", key=f"delete_{i}"):
                    get_response(f"tasks/{i}", "DELETE")
                    st.experimental_rerun()
        else:
            st.write("No tasks found.")

    # Add Task
    st.subheader("Add Task")
    task_description = st.text_input("Task Description")
    if st.button("Create Task"):
        task_data = {"description": task_description}
        response = get_response("tasks", "POST", task_data)
        st.success(response.get("message", "Task created!"))
        st.experimental_rerun()

# Meetings Tab
with tabs[1]:
    st.header("Meetings")

    # Fetch meetings
    meetings_data = get_response("meetings", "GET")
    if "meetings" in meetings_data:
        meetings = meetings_data["meetings"]
        if meetings:
            st.subheader("Upcoming Meetings")
            for i, meeting in enumerate(meetings):
                st.write(
                    f"{i + 1}. {meeting['title']} on {meeting['date']} at {meeting['time']} - {meeting['agenda']} "
                    f"Recording: {'Yes' if meeting['is_recording'] else 'No'}"
                )
                if st.button(f"Edit Meeting", key=f"edit_{i}"):
                    meeting_data = {
                        "title": st.text_input("Title", meeting["title"], key=f"title_{i}"),
                        "date": st.date_input("Date", key=f"date_{i}"),
                        "time": st.time_input("Time", key=f"time_{i}"),
                        "agenda": st.text_area("Agenda", meeting["agenda"], key=f"agenda_{i}"),
                        "is_recording": st.checkbox("Recording", value=meeting["is_recording"], key=f"recording_{i}")
                    }
                    if st.button("Save Meeting", key=f"save_{i}"):
                        get_response(f"meetings/{i}", "PUT", meeting_data)
                        st.experimental_rerun()
                if st.button(f"Delete Meeting", key=f"delete_meeting_{i}"):
                    get_response(f"meetings/{i}", "DELETE")
                    st.experimental_rerun()
        else:
            st.write("No meetings found.")

    # Add Meeting
    st.subheader("Add Meeting")
    meeting_title = st.text_input("Title")
    meeting_date = st.date_input("Date")
    meeting_time = st.time_input("Time")
    meeting_agenda = st.text_area("Agenda")
    meeting_is_recording = st.checkbox("Recording")
    if st.button("Create Meeting"):
        meeting_data = {
            "title": meeting_title,
            "date": str(meeting_date),
            "time": str(meeting_time),
            "agenda": meeting_agenda,
            "is_recording": meeting_is_recording,
        }
        response = get_response("meetings", "POST", meeting_data)
        st.success(response.get("message", "Meeting created!"))
        st.experimental_rerun()

# Chat Tab
with tabs[2]:
    st.header("Chat with Virtual Manager")

    # Chat Interface
    user_prompt = st.text_area("Enter your query:")
    if st.button("Send"):
        chat_data = {"prompt": user_prompt}
        response = get_response("chat", "POST", chat_data)
        if "response" in response:
            st.text_area("Response", value=response["response"], height=200)
        else:
            st.error(response.get("error", "Error communicating with backend."))
